# Scrap My Prop

### Laboratórios de Engenharia Informática

**"Development of an IT solution for the extraction and automatic analysis of data and relevant information for the calculation of land and properties."**

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from IPython.core.debugger import set_trace
import geopy.distance
import os.path
from sklearn import metrics

In [ ]:
#data_imo = pd.read_csv('../dados/dados_imovirtual_25_05.csv', engine='python', encoding='utf8')
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 10)
#data_imo.head()

### Id

In [ ]:
def preparationData(data_imo):
    data_imo = data_imo[['Id','Preço']]
    
    data_imo = data_imo.dropna(subset=['Id'])
    data_imo.index = np.arange(1, len(data_imo) + 1)
    
    data_imo['Id'] = data_imo['Id'].astype(int)
    
    return data_imo

### Preço

In [ ]:
def toNumeric(string):
    res = str(string)
    res = res.replace(" ", "")
    res = res.replace(",", ".")
    res = float(pd.to_numeric(res, errors='ignore')) # tem que ser float porque esse tipo consegue interpretar o np.nan
    return res

In [ ]:
def preparationPreco(data_imo):

    data_imo['Preço'] = data_imo['Preço'].apply(toNumeric)

    nan_prices = data_imo['Preço'].index[data_imo['Preço'].apply(np.isnan)]
    data_imo = data_imo.drop(nan_prices)
    data_imo.index = np.arange(1, len(data_imo) + 1)

    data_imo['Preço'] = data_imo['Preço'].apply(int)
    
    return data_imo

In [ ]:
class Imovel():
    def getData(self):
        return self.data

    def getPreco(self):
        return self.preco

    def __init__(self, data, preco):
        self.data = data
        self.preco = preco

In [ ]:
#imoveis = {}

#dia = 25

#for index, row in data_imo.iterrows():
    #print(row['Preço'], row['Id'], index)
    #imovel = Imovel(f'{dia}/05', row['Preço'])
    #imoveis[row['Id']] = imovel

In [ ]:
#imoveis.get(15334397).preco

In [ ]:
imoveis = {}

for mes in range(4,7):
    if mes > 0 and mes < 10:
        mes = str(mes).zfill(2)
    print("------ Mês:", mes)
    for dia in range(1,31):
        if dia > 0 and dia < 10:
            dia = str(dia).zfill(2)
        print("--- Dia:", dia)
        if os.path.isfile(f'../dados/dados_imovirtual_{dia}_{mes}.csv'):
            data_imo = pd.read_csv(f'../dados/dados_imovirtual_{dia}_{mes}.csv', engine='python', encoding='utf8')
            data_imo = preparationData(data_imo)
            data_imo = preparationPreco(data_imo)
            print(data_imo.shape)
            for index, row in data_imo.iterrows():
                imovel = Imovel(f'{dia}/{mes}', row['Preço'])
                if row['Id'] in imoveis:
                    imoveis[row['Id']].append(imovel)
                else:
                    imoveis[row['Id']] = [imovel]
                    

In [ ]:
imoveis.keys()

In [ ]:
x = []
y = []

for i in imoveis.get(15355922):
    x.append(i.data)
    y.append(i.preco)
    
#max(y)
#min(y)

In [ ]:
plt.figure(figsize=(20, 8))
plt.xticks(rotation=90)
plt.plot(x, y, 'go--', linewidth=2, markersize=12)
plt.show()